In [4]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='D:\Temp\MachineLearning\data\cifar-10-batches-py'

class   CifarData:
    def __init__(self,filelist,shuffle):
        all_data=[]
        all_labels=[]
        #循环读取多个批次的数据集文件
        for filename in filelist:
            data,labels=self.load_data(filename)      
            all_data.append(data)
            all_labels.append(labels)
        # 将所有data以纵向的方式合并到_data
        self._data=np.vstack(all_data)
        # 数据归一化处理
        self._data=self._data/127.5 - 1
        # 将所有label以横向方式合并到_label
        self._labels=np.hstack(all_labels)
        # 数据中样本个数
        self._example_num=self._data.shape[0]
        print(self._example_num)
        # 定位数据集遍历的位置指针
        self._indicator=0
        # 根据传入的shuffle决定是否对数据进行打乱
        self._shuffle=shuffle
        if self._shuffle:
            self.shuffle_data()
                 
    def load_data(self,filename):
        """读取一个批文件中的数据"""
        with open(filename,'rb') as f:
            data=pickle.load(f,encoding='bytes')
            return data[b'data'],data[b'labels']
        
    def shuffle_data(self):
        # 返回一个打乱的序号列表，例如将0~5 随机打乱为 [5,3,2,4,0,1]
        shuffle_list=np.random.permutation(self._example_num)
        # 根据随机列表重排数据和对应的标签值
        self._data=self._data[shuffle_list]
        self._labels=self._labels[shuffle_list]
        
    def next_batch(self,batch_size ):
        '''返回当前指针self._indicator之后batch_size个数据'''
        end_indicator=self._indicator+batch_size
        # 如果相加结果超过了样本个数
        if end_indicator>self._example_num:
            # 如果可以洗牌，打乱数据并从0开始重新取数据
            if self._shuffle:
                self.shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("没有更多数据了！")    
        # 返回从_indicator到end_indicator的data和labels，并将指针后移到end_indicator
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels

# 拼接训练数据的路径名        
train_file=[os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range (1,6)]
test_file=[os.path.join(CIFAR_DIR,'test_batch')]
# 创建训练数据对象
train_data=CifarData(train_file,True)     

x=tf.placeholder(tf.float32,[None,3072])
y=tf.placeholder(tf.int64,[None])

# 通过封装的函数构建多层神经网络
layer1=tf.layers.dense(x,100,activation=tf.nn.relu)
layer2=tf.layers.dense(layer1,50,activation=tf.nn.relu)
y_predict=tf.layers.dense(layer2,10)

# 使用交叉熵作为损失函数
loss=tf.losses.sparse_softmax_cross_entropy(y,y_predict)

# 将标签预测向量中最大的下标作为预测值，例如[0.1,0.8...0.01]，则预测为第二类
predict=tf.math.argmax(y_predict,1)
# 通过equal函数逐一比较predict，y_reshape的每一个元素
correction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correction,tf.float64))
#定义优化方法
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init=tf.global_variables_initializer()
batch_size=20
train_steps=10000
test_steps=100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        if i % 500==0:
            print("第%d步：损失：%.5f，精确度：%.5f"%(i,loss_val,acc_val))
        #每训练1000次，在test数据集上进行一次测试
        if i%1000==0:
            test_data=CifarData(test_file,False)   
            all_acc=[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
                test_acc=sess.run([accuracy],feed_dict={x:test_batch_data,y:test_batch_labels})
                all_acc.append(test_acc)
            test_acc_mean=np.mean(all_acc)
            print("第%d步测试集准确率%.4f"%(i,test_acc_mean))

50000
第0步：损失：2.46472，精确度：0.05000
10000
第0步测试集准确率0.1220
第500步：损失：1.45059，精确度：0.30000
第1000步：损失：1.90058，精确度：0.35000
10000
第1000步测试集准确率0.4035
第1500步：损失：1.74807，精确度：0.50000
第2000步：损失：1.62755，精确度：0.45000
10000
第2000步测试集准确率0.4395
第2500步：损失：2.01486，精确度：0.15000
第3000步：损失：1.85604，精确度：0.30000
10000
第3000步测试集准确率0.4545
第3500步：损失：1.56555，精确度：0.30000
第4000步：损失：1.83965，精确度：0.30000
10000
第4000步测试集准确率0.4605
第4500步：损失：1.93425，精确度：0.40000
第5000步：损失：1.51490，精确度：0.40000
10000
第5000步测试集准确率0.4855
第5500步：损失：1.59734，精确度：0.45000
第6000步：损失：1.47874，精确度：0.45000
10000
第6000步测试集准确率0.5045
第6500步：损失：2.01359，精确度：0.30000
第7000步：损失：1.27393，精确度：0.45000
10000
第7000步测试集准确率0.4910
第7500步：损失：1.63198，精确度：0.30000
第8000步：损失：1.17629，精确度：0.55000
10000
第8000步测试集准确率0.4895
第8500步：损失：1.76396，精确度：0.50000
第9000步：损失：1.44709，精确度：0.55000
10000
第9000步测试集准确率0.4910
第9500步：损失：1.48972，精确度：0.40000
